In [3]:
import sys

sys.path.append('/mnt/home/tudomlumleart/.local/lib/python3.10/site-packages')
sys.path.append('/mnt/home/tudomlumleart/.local/lib64/python3.10/site-packages')
sys.path.append('/mnt/sw/nix/store/gpkc8q6zjnp3n3h3w9hbmbj6gjbxs85w-python-3.10.10-view/lib/python3.10/site-packages')
sys.path.append('/mnt/home/tudomlumleart/00_VirtualEnvironments/jupyter-gpu/lib/python3.10/site-packages')
sys.path.append('/mnt/home/tudomlumleart/00_VirtualEnvironments/jupyter-gpu/lib64/python3.10/site-packages')

import jax
import jax.typing
import jax.numpy as jnp
import jax.scipy.stats as stats
import blackjax
import numpy as np
import matplotlib.pyplot as plt


# Define parameters
num_monomers = 1000  # Number of monomers in the polymer
temperature = 1.0    # Temperature for the simulation

# Initialize polymer configuration
def initialize_polymer(num_monomers):
    positions = np.zeros((num_monomers, 3))
    for i in range(1, num_monomers):
        step = np.random.choice([-1, 1], size=3)
        positions[i] = positions[i-1] + step
    return positions

# Define the energy function for the fractal globule
def fractal_globule_energy(positions):
    distances = jnp.linalg.norm(positions[:, None, :] - positions[None, :, :], axis=-1)
    energy = jnp.sum(jnp.exp(-distances))
    return energy

# Initialize polymer configuration
key = jax.random.PRNGKey(0)
initial_positions = initialize_polymer(num_monomers)

# Set up the HMC sampler
def log_posterior(positions):
    return -fractal_globule_energy(positions) / temperature

initial_positions = jnp.array(initial_positions)
hmc = blackjax.hmc(log_posterior, step_size=1e-2, inverse_mass_matrix=jnp.ones((num_monomers, 3)))

# Run the HMC sampler
key, subkey = jax.random.split(key)
initial_state = hmc.init(initial_positions, subkey)
kernel = jax.jit(hmc.step)

# Perform sampling
n_samples = 1000
states = [initial_state]
for _ in range(n_samples):
    key, subkey = jax.random.split(key)
    states.append(kernel(subkey, states[-1]))

# Extract samples
samples = jnp.array([state.position for state in states])

# Visualize the last sampled configuration
final_sample = samples[-1]
plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')
ax.plot3D(final_sample[:, 0], final_sample[:, 1], final_sample[:, 2], 'blue')
plt.show()


AttributeError: module 'jax' has no attribute 'typing'

In [8]:
jax.tree

AttributeError: module 'jax' has no attribute 'tree'

In [1]:
import torch


In [2]:
torch.cuda.is_available()

True